In [2]:
import nltk
import re
import pickle
import numpy as np
from keras.models import load_model
import json
import random

def ajusta_texto( texto ):
    texto = texto.lower()
    texto = re.sub(r"[^a-zA-ZÀ-ú\s]", "", texto)
    return texto

def load_data():
    global model, intents, words, classes
    model = load_model('chatbot_model.h5')
    intents = json.loads(open('intents.json').read())
    words = pickle.load(open('words.pkl','rb'))
    classes = pickle.load(open('classes.pkl','rb'))

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [ajusta_texto(word) for word in sentence_words]
    return sentence_words
    
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
    
def predict_class(sentence):
    global model, intents, words, classes
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]

    # Refinamento da aceitação das respostas prováveis: Padrão = mínimo 70% de probabilidade
    ERROR_THRESHOLD = 0.7

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
    
def getResponse(ints):
    global intents
    tag = ints[0]['intent']
    list_of_intents = intents['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text)
    res = "Não compreendi a pergunta. Tente especificar melhor."
    if len(ints) > 0:
        res = getResponse(ints)
    response = {'response':res, 'predictions':ints, 'question':text}
    return json.dumps(response)

load_data()


chatbot_response("quais?")
    

1/1 [==============================] - 0s 92ms/step


'{"response": "N\\u00e3o compreendi a pergunta. Tente especificar melhor.", "predictions": [], "question": "quais?"}'